# Question: What is the stratigraphy of the Black Widow Field in US GOM?

We are often asked questions about the subsurface that we try to approximate instead of getting the true value.  The problem isn't that we don't know how to answer it, the problem is that the data is in such a mess that it will take an inordinate amount of time to find the answer.

In this notebook imagine that you've been asked to understand the stratigraphic distribution of a field you have no previous experience with.  You have two options:
1. Create a nDI project with wells, tops, seismic, horizons, etc. and piece together the story.
2. Dig into the biostratigraphic data which has everything you need but is a pain to deal with.

Luckily, Python can help us with Option 2.  Let's explore a solution.

![alt text](../images/black_widow.png)

## Step 1: Load and Inspect the Data
Our data manager has given us a list of well API's for the Black Widow Field which we load into the notebook.  This is called a list.

We know that all the biostratigraphic data we need is on the BOEM webiste:
https://www.data.boem.gov/Main/Paleo.aspx#ascii

We can download an txt file with these data.  Let's first load it into Pandas.

In [1]:
#Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_colwidth', 140)
from matplotlib.colors import ListedColormap

In [ ]:
#list of well API's from Black Widow Field

black_widow_wells = [608104004700, 608104010000, 608104010001, 608105002500, 608105002501, 608105006200, 608105006201, 608105006202, 608105007900, 608105007901, 608105007902,
                      608105007903, 608105008000, 608105008001, 608105008002, 608105008003, 608105008004, 608105008005, 608105008006, 608105008007, 608105008008,
                     608105008009, 608105008100, 608105008101, 608105008102, 608105008103, 608105008104, 608105008700, 608105009500, 608105009600,
                     608105009601, 608105009800, 608105009801, 608105009900, 608105009901, 608105010000, 608105010001]

In [ ]:
#Load and inspect data
mess = pd.read_csv('../data/gepaldmp_Paleo.txt', header=None)
mess.head()

This is a __mess__, but not impossible.  The header information, foundon the website, for the table shows that the data is organized by start position, character length.  We can use this information to break out the data.

![alt text](../images/messy_data.png "Header Information for Biostratigraphy Table")

## Step 2. Clean and Organize data

To do the breakout we will use indexing which is part of core Python which allows a user to specify ranges of data to pull out and manipulate.  Each piece of data has an index that starts with 0.  In the example below we index the word "geology":
![alt text](../images/geology.png)

In [ ]:
#example of indexing, 
target = 'geology'

print(target[0:3])
print(target[3:7])

Using the header ranges we can build a dataframe with all the data sepatated out.

In [ ]:
#Clean up table with indexing 12 separate columns

#create empty dataframe
df = pd.DataFrame()

#Index the  columns from BOEM document instructions
df['Point']=mess[0].str[0:1].astype(str)
df['API']=mess[0].str[1:13].astype(np.int64)
df['Report_ID']=mess[0].str[13:15].astype(int)
df['Total_Report']=mess[0].str[15:17].astype(int)
df['MD']=mess[0].str[17:22].astype(int)
df['TVD']=mess[0].str[22:27].astype(int)
df['Conf_PALEO']=mess[0].str[27:30].astype(str)
df['AT_IN_PALEO']=mess[0].str[30:32].astype(str)
df['PALEO']=mess[0].str[32:132].astype(str)
df['Conf_ECO']=mess[0].str[132:135].astype(str)
df['AT_IN_ECO']=mess[0].str[135:137].astype(str)
df['ECO']=mess[0].str[137:138].astype(str)
df['PALEO_clean'] = df['PALEO'].str.strip() #extra step to remove whitespace

In [ ]:
#Show first 10 rows
df.head(10)

## Step 3. Filtering and add columns
Looking over the first 10 rows of the dataframe we notice some data that isn't useful for our goal or is missing:
1. We only want the biostratigraphic samples, not the mentions of "salt" or "first sample examined".  _Let's filter those out._
2. The "PALEO_clean" column has lots of information in it (Epoch, Age, species).  _Let's separate those out into separate columns._
3. An Age Millions of Years column would be useful for plotting. _Let's add a new column with dates._

Even from our limited view we notice that if a column has bug information it will start with the Epoch and whether it is Upper, Middle, or Lower.  We can use these key words to filter just to the bugs.

In [ ]:
#Filter to rows containing "Upper, Middle, or Lower"
only_bugs = df[(df['PALEO_clean'].str.contains("Lower"))|(df['PALEO_clean'].str.contains("Upper"))|
         (df['PALEO_clean'].str.contains("Middle"))]

Next wek want to split out the "PALEO_clean" column so we have more columns to filter on.  To do so we'll create a loop that will go through each row and split data when it sees either a "(" or ")".  The loop will also rename the columns based on a list we've generated.  Lastly we'll drop redundant columns.

In [ ]:
#Names of new columns we would like to add
new_cols = ['Epoch','Age', 'Species']

#Loop to split names
only_bugs_split = only_bugs.copy()
for i in range(0,3):
    only_bugs_split[new_cols[i]]=only_bugs['PALEO_clean'].str.split(pat='\(|\)', expand=True)[i]

#Remove redundant columns and inspect split table
only_bugs_split=only_bugs_split.drop(['PALEO', 'PALEO_clean'], axis=1) 
only_bugs_split.head(10)

### Using Dictionaries
For the final step we want to add a numerica age to the table.  Since geologic ages are well publicized we only need to match the age to a defined number.  To do so we'll use another python data object called a dictionary.

Python dictionaries are a data type that allows you to store a variety of informaiton so that it can be accessed later by different commands.  Here's an example about my favorite dinosaur: Triceratops.

![alt text](../images/triceratops.png)

In [ ]:
#Here's a premade dictionary that describes the youngest data for the ages from Jurassic to Present

ages_dictionary = {'Albian': 100.5, 'Aptian': 113.0, 'Aquitanian': 20.44, 'Barremian': 125.0, 'Bartonian': 37.8, 'Berriasian': 139.8, 'Burdigalian': 15.97,
                    'Calabrian': 0.012, 'Callovian': 163.5, 'Campanian': 72.1, 'Cenomanian': 93.9, 'Chattian': 23.03, 'Coniacian': 86.3, 'Danian': 61.6,
                   'Gelasian': 1.8, 'Hauterivian': 129.4, 'Ionian': 0.7809999999999999, 'Kimmeridgian': 152.1, 'Langhian': 13.82, 'Lutetian': 41.2, 'Maastrichtian': 66.0,
                    'Messinian': 5.33, 'Oxfordian': 157.3, 'Piacenzian': 2.58, 'Priabonian': 33.9, 'Rupelian': 27.82, 'Santonian': 83.6, 'Selandian': 59.2,
                    'Serravallian': 11.63, 'Thanetian': 56.0, 'Tithonian': 145.0, 'Tortonian': 7.246, 'Turonian': 89.8, 'Valanginian': 132.9, 'Ypresian': 47.8, 'Zanclian': 3.6}

In [ ]:
#You can do a lot with dictionaries including, looking up values
ages_dictionary['Albian']

In [ ]:
#Map Ages to Ages MA using the dictionary
only_bugs_ages = only_bugs_split.copy()
only_bugs_ages['Age_MA']=only_bugs_ages['Age'].map(ages_dictionary)
only_bugs_ages.head()

## Step 4. Understanding the Black Widow Field

Now that we have our data the way we want it we can start with the analysis.

The first question is, how are the ages distributed across the field? Let's use a histogram.

In [ ]:
#Filter to only wells in Black Widow area
black_widow_bugs = only_bugs_ages.loc[only_bugs_ages['API'].isin(black_widow_wells)]

#plot histogram 
fig = plt.subplots(figsize=(10,6))
plt.hist(black_widow_bugs['Age_MA'].values, bins=100)
plt.xlabel('Age (MA)')
plt.title ('Histogram of Biostratigraphic Ages in Black Widow Field')
plt.show()

The majority of samples are <5 Ma, this is a Pliocene Field, but why are the samples from the Early Cenozoic and Mesozoic?
### Box Plots by Well and Age
Box plots so distributions betters so let's build one with wells on the x axis.  Let's also add the points so that we can see how the samples plot.

In [ ]:
#Box Plots by well
f, ax = plt.subplots(figsize=(20,8))
sns.boxplot(x="API", y="Age_MA", data=black_widow_bugs)
sns.swarmplot(x="API", y="Age_MA", data=black_widow_bugs,
             size=3, color='black', linewidth=0)
ax.invert_yaxis()

Looks like only one well is causing the anomalous values (608105008700).   Digging a bit deeper you would find that this well is called "Chin Music" and is famous in the GOM for being a strange well, not just because it found a Mesozoic section but also the way it is ordered.

Let's plot ages with depth to see.

In [ ]:
chin_music = black_widow_bugs.query("API==608105008700")
my_cmap=ListedColormap(sns.color_palette("Paired", 8))

fig = plt.subplots(figsize=(10,6))
plt.scatter(chin_music.Age_MA.values, chin_music.TVD.values, c=chin_music.Age_MA.values, s=70,cmap=my_cmap)
plt.ylim(17000,6000)
plt.xlabel('Age (MA)')
plt.ylabel('Depth (TVD)')
plt.xticks(rotation=90)
plt.show()

__That's weird!__  The shallow section starts in the Pliocene and progressively gets older.  Then at ~12,500 the well encounters Jurassic then Cretaceous then Paleogene, and finall TD's in the Miocene.  The stratigraphy is inverted.  In the GOM this is known as a Megaflap!

### Enviornment of Deposition 
Another element in the biostratigraphic database is ecozone classification (i.e. 1=Inner Shelf, 7=Abyssal).  Plotting them as a scatter of Age MA vs. Ecozone will inform us how environments have changed through time.

In [ ]:
#Scatter Grid plot of Ecozones per well.

bw_eco_bugs = black_widow_bugs.copy()
bw_eco_bugs['ECO']=pd.to_numeric(bw_eco_bugs['ECO'], errors='coerce').dropna().astype(int) #remove rows where no ecozone data present

g = sns.FacetGrid(bw_eco_bugs, col="API", hue="ECO", palette="YlGnBu",
                     col_wrap=4, height=4.5)
g = g.map(plt.scatter, "Age_MA", 'ECO')

Here we can see that majority of the wells in the Black Window field are in Ecozones 4-5, implying deeper water deposition.  Only Chin Music shows a variety where the Jurassic was deposited in Ecozone 2 (inner neritic) and progressively gets deeper through time.

### Stratigraphic Table
Finally, we can use Pandas groupby function to create a table that clearly displays ages and bugs.

In [ ]:
#Hierarchcal Table of bugs

bug_table = black_widow_bugs[['Epoch', 'Age', 'Species', 'Age_MA']]
bug_table = bug_table.groupby(['Epoch', 'Age', 'Species']).min()
bug_table.sort_values('Age_MA')